# 08: SparkSQL

Originally, SparkSQL was an extension of RDDs with the concept of a `DataFrame` that adds a "schema" for records, defined using Scala _case classes_, tuples, or a built-in schema mechanism. The DataFrame API is inspired by similar `DataFrame` concepts in R and Python libraries. The transformation and action steps written in any of the support languages, as well as SQL queries embedded in strings, are translated to the same, performant query execution model, optimized by a new query engine called *Catalyst*.

The even newer `Dataset` API encapsulates `DataFrame`, but adds more type safety for the data columns. We'll stick with the `DataFrame` here.

> **Tip:** Even if you prefer the Scala collections-like `RDD` API, I recommend using the `DataFrame` API, because the performance is _significantly_ better in most cases, due to internal optimizations.

Furthermore, SparkSQL has convenient support for reading and writing files encoded using [Parquet](http://parquet.io), [ORC](https://orc.apache.org/), JSON, CSV, etc.

Finally, SparkSQL embeds access to a Hive _metastore_, so you can create and delete tables, and run queries against them using SparkSQL.

This example treats the KJV text we've been using as a table with a schema. It runs several SQL queries on the data, then performs the same calculation using the `DataFrame` API.

See the corresponding Spark job [SparkSQL8.scala](https://github.com/deanwampler/spark-scala-tutorial/blob/master/src/main/scala/sparktutorial/SparkSQL8.scala) and the "script" suitable for _spark-shell_, [SparkSQL8-script.scala](https://github.com/deanwampler/spark-scala-tutorial/blob/master/src/main/scala/sparktutorial/SparkSQL8-script.scala), because SQL queries are nice to use interactively!

In [1]:
val in = "../data/kjvdat.txt"                       // '|' separated
val abbrevToNames = "../data/abbrevs-to-names.tsv"  // tab separated

in = ../data/kjvdat.txt
abbrevToNames = ../data/abbrevs-to-names.tsv


../data/abbrevs-to-names.tsv

This time, we won't use the `toText` method we defined before, we'll use a big regex and do pattern matching with it.

In [2]:
val lineRE = """^\s*([^|]+)\s*\|\s*([\d]+)\s*\|\s*([\d]+)\s*\|\s*(.*)~?\s*$""".r

lineRE = ^\s*([^|]+)\s*\|\s*([\d]+)\s*\|\s*([\d]+)\s*\|\s*(.*)~?\s*$


^\s*([^|]+)\s*\|\s*([\d]+)\s*\|\s*([\d]+)\s*\|\s*(.*)~?\s*$

Let's define a case class `Verse` to represent our records.

In [3]:
case class Verse(book: String, chapter: Int, verse: Int, text: String)

defined class Verse


Now load and parse the data. Note that using `flatMap` effective removes the bad records, for which we return `Nil`, while we return `Seq(verse)` on success.

In [4]:
val versesRDD = sc.textFile(in).flatMap {
  case lineRE(book, chapter, verse, text) =>
    Seq(Verse(book, chapter.toInt, verse.toInt, text))
  case line =>
    Console.err.println(s"Unexpected line: $line")
    Nil // or use Seq.empty[Verse]. It will be eliminated by flattening.
}

versesRDD = MapPartitionsRDD[2] at flatMap at <console>:33


MapPartitionsRDD[2] at flatMap at <console>:33

Now create a `DataFrame` from this `RDD` and create a temporary "view". We're going to reuse this data over and over, so now it's very useful to _cache_ it in memory.

In [5]:
val verses = spark.createDataFrame(versesRDD)
verses.createOrReplaceTempView("kjv_bible")
verses.cache()

verses = [book: string, chapter: int ... 2 more fields]


[book: string, chapter: int ... 2 more fields]

Print the first 20 lines (the default), or pass an integer argument to show a different number
of lines, as here:

In [6]:
verses.show(10)

+----+-------+-----+--------------------+
|book|chapter|verse|                text|
+----+-------+-----+--------------------+
| Gen|      1|    1|In the beginning ...|
| Gen|      1|    2|And the earth was...|
| Gen|      1|    3|And God said, Let...|
| Gen|      1|    4|And God saw the l...|
| Gen|      1|    5|And God called th...|
| Gen|      1|    6|And God said, Let...|
| Gen|      1|    7|And God made the ...|
| Gen|      1|    8|And God called th...|
| Gen|      1|    9|And God said, Let...|
| Gen|      1|   10|And God called th...|
+----+-------+-----+--------------------+
only showing top 10 rows



Pass an optional `truncate = false` argument for wider output.

In [7]:
verses.show(5, truncate = false)

+----+-------+-----+-----------------------------------------------------------------------------------------------------------------------------------------------+
|book|chapter|verse|text                                                                                                                                           |
+----+-------+-----+-----------------------------------------------------------------------------------------------------------------------------------------------+
|Gen |1      |1    |In the beginning God created the heaven and the earth.~                                                                                        |
|Gen |1      |2    |And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.~|
|Gen |1      |3    |And God said, Let there be light: and there was light.~                                                                                        |
|Gen |1   

Now write some SQL queries, with SQL and with the Dataset/DataFrame API!

In [8]:
val godVerses = spark.sql("SELECT * FROM kjv_bible WHERE text LIKE '%God%'")

godVerses = [book: string, chapter: int ... 2 more fields]


[book: string, chapter: int ... 2 more fields]

In [9]:
println("Number of verses that mention God: "+godVerses.count())
godVerses.show(truncate=false)

Number of verses that mention God: 3585
+----+-------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|book|chapter|verse|text                                                                                                                                                                                                  |
+----+-------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Gen |1      |1    |In the beginning God created the heaven and the earth.~                                                                                                                                               |
|Gen |1      |2    |And the earth was without form, and void; and darkness was u

In [10]:
println("The query plan:")
godVerses.queryExecution   // Compare with godVerses.explain(true)

The query plan:


      +- SerializeFromObject [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, assertnotnull(assertnotnull(input[0, Verse, true])).book, true, false) AS book#5, assertnotnull(assertnotnull(input[0, Verse, true])).chapter AS chapter#6, assertnotnull(assertnotnull(input[0, Verse, true])).verse AS verse#7, staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, assertnotnull(assertnotnul...


== Parsed Logical Plan ==
'Project [*]
+- 'Filter 'text LIKE %God%
   +- 'UnresolvedRelation `kjv_bible`

== Analyzed Logical Plan ==
book: string, chapter: int, verse: int, text: string
Project [book#5, chapter#6, verse#7, text#8]
+- Filter text#8 LIKE %God%
   +- SubqueryAlias kjv_bible
      +- SerializeFromObject [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, assertnotnull(assertnotnull(input[0, $line23.$read$$iw$$iw$Verse, true])).book, true, false) AS book#5, assertnotnull(assertnotnull(input[0, $line23.$read$$iw$$iw$Verse, true])).chapter AS chapter#6, assertnotnull(assertnotnull(input[0, $line23.$read$$iw$$iw$Verse, true])).verse AS verse#7, staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, assertnotnull(assertnotnull(input[0, $line23.$read$$iw$$iw$Verse, true])).text, true, false) AS text#8]
         +- ExternalRDD [obj#4]

== Optimized Logical Plan ==
Filter (isnotnull(text#8) && Contains(text#8, God))


Now using the API:

In [11]:
val godVersesDF = verses.filter(verses("text").contains("God"))
println("Number of verses that mention God: "+godVersesDF.count())
godVersesDF.show()

Number of verses that mention God: 3585
+----+-------+-----+--------------------+
|book|chapter|verse|                text|
+----+-------+-----+--------------------+
| Gen|      1|    1|In the beginning ...|
| Gen|      1|    2|And the earth was...|
| Gen|      1|    3|And God said, Let...|
| Gen|      1|    4|And God saw the l...|
| Gen|      1|    5|And God called th...|
| Gen|      1|    6|And God said, Let...|
| Gen|      1|    7|And God made the ...|
| Gen|      1|    8|And God called th...|
| Gen|      1|    9|And God said, Let...|
| Gen|      1|   10|And God called th...|
| Gen|      1|   11|And God said, Let...|
| Gen|      1|   12|And the earth bro...|
| Gen|      1|   14|And God said, Let...|
| Gen|      1|   16|And God made two ...|
| Gen|      1|   17|And God set them ...|
| Gen|      1|   18|And to rule over ...|
| Gen|      1|   20|And God said, Let...|
| Gen|      1|   21|And God created g...|
| Gen|      1|   22|And God blessed t...|
| Gen|      1|   24|And God said, Le

godVersesDF = [book: string, chapter: int ... 2 more fields]


[book: string, chapter: int ... 2 more fields]

Use `GROUP BY` and column aliasing

In [12]:
val counts = spark.sql("SELECT book, COUNT(*) as count FROM kjv_bible GROUP BY book")
counts.show(100)  // print the 1st 100 lines, but there are only 66 books/records...

+----+-----+
|book|count|
+----+-----+
| Sa1|  810|
| Ecc|  222|
| Mic|  105|
| Ti2|   83|
| Job| 1070|
| Th1|   89|
| Jer| 1364|
| Ezr|  280|
| Phi|  104|
| Rut|   85|
| Joe|   73|
| Gal|  149|
| Th2|   47|
| Jo2|   13|
| Mal|   55|
| Luk| 1151|
| Pro|  915|
| Jdg|  618|
| Nah|   47|
| Jde|   25|
| Heb|  303|
| Mat| 1071|
| Kg1|  816|
| Kg2|  719|
| Col|   95|
| Ti1|  113|
| Neh|  406|
| Rev|  404|
| Sol|  117|
| Ch2|  822|
| Num| 1288|
| Jam|  108|
| Hos|  197|
| Jo1|  105|
| Rom|  433|
| Deu|  959|
| Gen| 1533|
| Pe1|  105|
| Ch1|  942|
| Joh|  879|
| Psa| 2461|
| Jos|  658|
| Hab|   56|
| Dan|  357|
| Plm|   25|
| Est|  167|
| Sa2|  695|
| Pe2|   61|
| Mar|  678|
| Hag|   38|
| Eph|  155|
| Exo| 1213|
| Co2|  257|
| Lam|  154|
| Tit|   46|
| Amo|  146|
| Jo3|   14|
| Zac|  211|
| Co1|  437|
| Isa| 1292|
| Zep|   53|
| Lev|  859|
| Oba|   21|
| Act| 1007|
| Eze| 1273|
| Jon|   48|
+----+-----+



counts = [book: string, count: bigint]


[book: string, count: bigint]

**Exercise**: Update the previous query to sort output by the book names, by the counts. For convenience, I've pasted in the same query (different variable name). How much overhead does this add?

In [ ]:
val sorted_counts = spark.sql("SELECT book, COUNT(*) as count FROM kjv_bible GROUP BY book")
sorted_counts.show(100)  // print the 1st 100 lines, but there are only 66 books/records...

Use `coalesce` when you have too many partitions, e.g., a small data set and the default number of partitions (200) is too large.

In [13]:
val counts1 = counts.coalesce(1)
val nPartitions  = counts.rdd.partitions.size
val nPartitions1 = counts1.rdd.partitions.size
println(s"counts.count (can take a while, # partitions = $nPartitions):")
println(s"result: ${counts.count}")
println(s"counts1.count (usually faster, # partitions = $nPartitions1):")
println(s"result: ${counts1.count}")

counts.count (can take a while, # partitions = 200):
result: 66
counts1.count (usually faster, # partitions = 1):
result: 66


counts1 = [book: string, count: bigint]
nPartitions = 200
nPartitions1 = 1


1

Now do `GROUP BY` with the DataFrame API.

In [14]:
val countsDF = verses.groupBy("book").count()
countsDF.show(20)
countsDF.count

+----+-----+
|book|count|
+----+-----+
| Sa1|  810|
| Ecc|  222|
| Mic|  105|
| Ti2|   83|
| Job| 1070|
| Th1|   89|
| Jer| 1364|
| Ezr|  280|
| Phi|  104|
| Rut|   85|
| Joe|   73|
| Gal|  149|
| Th2|   47|
| Jo2|   13|
| Mal|   55|
| Luk| 1151|
| Pro|  915|
| Jdg|  618|
| Nah|   47|
| Jde|   25|
+----+-----+
only showing top 20 rows



countsDF = [book: string, count: bigint]


66

**Exercise**: Add sorting by book names and by the counts, using the API. For convenience, I've pasted in the same query (different variable name).

In [ ]:
val countsDF = verses.groupBy("book").count()
countsDF.show(20)
countsDF.count

Aggregations, like in Data Warehousing. We need to import some functions first:

In [15]:
import org.apache.spark.sql.functions._    // for min, max, etc.

In [16]:
verses.groupBy("book").agg(
  max(verses("chapter")),
  max(verses("verse")),
  count(verses("*"))
).sort($"count(1)".desc, $"book").show(100)

+----+------------+----------+--------+
|book|max(chapter)|max(verse)|count(1)|
+----+------------+----------+--------+
| Psa|         150|       176|    2461|
| Gen|          50|        67|    1533|
| Jer|          52|        64|    1364|
| Isa|          66|        38|    1292|
| Num|          36|        89|    1288|
| Eze|          48|        63|    1273|
| Exo|          40|        51|    1213|
| Luk|          24|        80|    1151|
| Mat|          28|        75|    1071|
| Job|          42|        41|    1070|
| Act|          28|        60|    1007|
| Deu|          34|        68|     959|
| Ch1|          29|        81|     942|
| Pro|          31|        36|     915|
| Joh|          21|        71|     879|
| Lev|          27|        59|     859|
| Ch2|          36|        42|     822|
| Kg1|          22|        66|     816|
| Sa1|          31|        58|     810|
| Kg2|          25|        44|     719|
| Sa2|          24|        51|     695|
| Mar|          16|        72|     678|


Alternative way of referencing columns in verses.

In [17]:
verses.groupBy("book").agg(
  max($"chapter"),
  max($"verse"),
  count($"*")
).sort($"count(1)".desc, $"book").show(100)

+----+------------+----------+--------+
|book|max(chapter)|max(verse)|count(1)|
+----+------------+----------+--------+
| Psa|         150|       176|    2461|
| Gen|          50|        67|    1533|
| Jer|          52|        64|    1364|
| Isa|          66|        38|    1292|
| Num|          36|        89|    1288|
| Eze|          48|        63|    1273|
| Exo|          40|        51|    1213|
| Luk|          24|        80|    1151|
| Mat|          28|        75|    1071|
| Job|          42|        41|    1070|
| Act|          28|        60|    1007|
| Deu|          34|        68|     959|
| Ch1|          29|        81|     942|
| Pro|          31|        36|     915|
| Joh|          21|        71|     879|
| Lev|          27|        59|     859|
| Ch2|          36|        42|     822|
| Kg1|          22|        66|     816|
| Sa1|          31|        58|     810|
| Kg2|          25|        44|     719|
| Sa2|          24|        51|     695|
| Mar|          16|        72|     678|


With just a single column, cube and rollup make less sense, but in a bigger dataset, you could do cubes and rollups, too.

In [18]:
verses.cube("book").agg(
  max($"chapter"),
  max($"verse"),
  count($"*")
).sort($"count(1)".desc, $"book").show(100)

+----+------------+----------+--------+
|book|max(chapter)|max(verse)|count(1)|
+----+------------+----------+--------+
|null|         150|       176|   31102|
| Psa|         150|       176|    2461|
| Gen|          50|        67|    1533|
| Jer|          52|        64|    1364|
| Isa|          66|        38|    1292|
| Num|          36|        89|    1288|
| Eze|          48|        63|    1273|
| Exo|          40|        51|    1213|
| Luk|          24|        80|    1151|
| Mat|          28|        75|    1071|
| Job|          42|        41|    1070|
| Act|          28|        60|    1007|
| Deu|          34|        68|     959|
| Ch1|          29|        81|     942|
| Pro|          31|        36|     915|
| Joh|          21|        71|     879|
| Lev|          27|        59|     859|
| Ch2|          36|        42|     822|
| Kg1|          22|        66|     816|
| Sa1|          31|        58|     810|
| Kg2|          25|        44|     719|
| Sa2|          24|        51|     695|


In [19]:
verses.rollup("book").agg(
  max($"chapter"),
  max($"verse"),
  count($"*")
).sort($"count(1)".desc, $"book").show(100)

+----+------------+----------+--------+
|book|max(chapter)|max(verse)|count(1)|
+----+------------+----------+--------+
|null|         150|       176|   31102|
| Psa|         150|       176|    2461|
| Gen|          50|        67|    1533|
| Jer|          52|        64|    1364|
| Isa|          66|        38|    1292|
| Num|          36|        89|    1288|
| Eze|          48|        63|    1273|
| Exo|          40|        51|    1213|
| Luk|          24|        80|    1151|
| Mat|          28|        75|    1071|
| Job|          42|        41|    1070|
| Act|          28|        60|    1007|
| Deu|          34|        68|     959|
| Ch1|          29|        81|     942|
| Pro|          31|        36|     915|
| Joh|          21|        71|     879|
| Lev|          27|        59|     859|
| Ch2|          36|        42|     822|
| Kg1|          22|        66|     816|
| Sa1|          31|        58|     810|
| Kg2|          25|        44|     719|
| Sa2|          24|        51|     695|


Map a field to a method to apply to it, but limited to at most one method per field.

In [20]:
verses.rollup("book").agg(Map(
  "chapter" -> "max",
  "verse" -> "max",
  "*" -> "count"
)).sort($"count(1)".desc, $"book").show(100)

+----+------------+----------+--------+
|book|max(chapter)|max(verse)|count(1)|
+----+------------+----------+--------+
|null|         150|       176|   31102|
| Psa|         150|       176|    2461|
| Gen|          50|        67|    1533|
| Jer|          52|        64|    1364|
| Isa|          66|        38|    1292|
| Num|          36|        89|    1288|
| Eze|          48|        63|    1273|
| Exo|          40|        51|    1213|
| Luk|          24|        80|    1151|
| Mat|          28|        75|    1071|
| Job|          42|        41|    1070|
| Act|          28|        60|    1007|
| Deu|          34|        68|     959|
| Ch1|          29|        81|     942|
| Pro|          31|        36|     915|
| Joh|          21|        71|     879|
| Lev|          27|        59|     859|
| Ch2|          36|        42|     822|
| Kg1|          22|        66|     816|
| Sa1|          31|        58|     810|
| Kg2|          25|        44|     719|
| Sa2|          24|        51|     695|


## Exercises

### Exercise 1: Try joins with the abbreviation data

See the project solution file [SparkSQL8-join-with-abbreviations-script.scala](https://github.com/deanwampler/spark-scala-tutorial/blob/master/src/main/scala/sparktutorial/solns/SparkSQL8-join-with-abbreviations-script.scala).

Here we set up the abbreviations, similar to the RDD Joins examples.

Now load the abbreviations, similar to how we loaded the Bible verses. First we need a case class...

In [21]:
case class Abbrev(abbrev: String, name: String)

defined class Abbrev


In [22]:
val abbrevNamesRDD = sc.textFile(abbrevToNames).flatMap { line =>
  val ary=line.split("\t")
  if (ary.length != 2) {
    Console.err.println(s"Unexpected line: $line")
    Nil // or use Seq.empty[Abbrev]. It will be eliminated by flattening.
  } else {
    Seq(Abbrev(ary(0), ary(1)))
  }
}

abbrevNamesRDD = MapPartitionsRDD[164] at flatMap at <console>:34


MapPartitionsRDD[164] at flatMap at <console>:34

In [23]:
val abbrevNames = spark.createDataFrame(abbrevNamesRDD)
abbrevNames.createOrReplaceTempView("abbrevs_to_names")

abbrevNames = [abbrev: string, name: string]


[abbrev: string, name: string]

### Exercise 2: Try other SQL constructs

Both using actual SQL and the API.